# 15-Minute Crypto Markets Analysis

Interactive charts for analyzing Polymarket 15-minute crypto prediction markets (BTC/ETH up or down).

**Filter:** Markets matching "Up or Down" pattern (e.g., "Bitcoin Up or Down - January 17, 10:30PM-10:45PM")

In [ ]:
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from supabase import create_client, Client

# Load environment variables
load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
print(f"Connected to Supabase: {SUPABASE_URL[:30]}...")

## 1. Discover Available 15m Markets

In [ ]:
# Step 1: Check what's in orderbook_snapshots
snapshots_sample = supabase.table("orderbook_snapshots").select("asset_id, market").limit(100).execute()
print(f"Orderbook snapshots returned: {len(snapshots_sample.data)} rows")

if snapshots_sample.data:
    asset_ids_with_data = list(set(row['asset_id'] for row in snapshots_sample.data))
    print(f"Unique asset_ids with data: {len(asset_ids_with_data)}")
    print(f"Sample asset_id: {asset_ids_with_data[0]}")
else:
    asset_ids_with_data = []
    print("No orderbook data found!")

# Step 2: Check markets table
markets_response = supabase.table("markets").select("*").limit(100).execute()
print(f"\nMarkets table returned: {len(markets_response.data)} rows")

if markets_response.data:
    all_markets_df = pd.DataFrame(markets_response.data)
    print(f"Sample token_id: {all_markets_df.iloc[0]['token_id']}")
    
    # Filter for 15m markets
    markets_df = all_markets_df[all_markets_df['question'].str.contains('Up or Down', case=False, na=False)]
    print(f"15m markets (Up or Down): {len(markets_df)}")
else:
    markets_df = pd.DataFrame()

# Step 3: Show available data
if not markets_df.empty:
    display_cols = ['question', 'outcome', 'token_id', 'state']
    available_cols = [c for c in display_cols if c in markets_df.columns]
    display(markets_df[available_cols].head(10))
elif not all_markets_df.empty:
    print("\nNo 15m markets, showing all markets:")
    display(all_markets_df[['question', 'outcome', 'token_id']].head(10))

In [ ]:
# Select a market/token to analyze
if not markets_df.empty:
    # Use the first token with data
    selected = markets_df.iloc[0]
    TOKEN_ID = selected['token_id']
    MARKET_QUESTION = selected.get('question', f'Token {TOKEN_ID[:30]}...')
    
    print(f"Selected: {MARKET_QUESTION}")
    print(f"Token ID: {TOKEN_ID}")
    if 'outcome' in selected:
        print(f"Outcome: {selected['outcome']}")
else:
    TOKEN_ID = "YOUR_TOKEN_ID_HERE"
    MARKET_QUESTION = "Manual Selection"
    print("No markets found. Set TOKEN_ID manually above.")

## 2. Load Orderbook Snapshots

In [ ]:
# Time range for analysis
HOURS_BACK = 4  # Adjust as needed
start_time = int((datetime.now() - timedelta(hours=HOURS_BACK)).timestamp() * 1000)

# Fetch orderbook snapshots
snapshots_response = supabase.table("orderbook_snapshots") \
    .select("*") \
    .eq("asset_id", TOKEN_ID) \
    .gte("timestamp", start_time) \
    .order("timestamp", desc=False) \
    .execute()

snapshots_df = pd.DataFrame(snapshots_response.data)
print(f"Loaded {len(snapshots_df)} orderbook snapshots")

if not snapshots_df.empty:
    snapshots_df['datetime'] = pd.to_datetime(snapshots_df['timestamp'], unit='ms')
    snapshots_df = snapshots_df.sort_values('timestamp')
    
    # Show forward-fill breakdown
    if 'is_forward_filled' in snapshots_df.columns:
        real_events = len(snapshots_df[snapshots_df['is_forward_filled'] == False])
        ff_events = len(snapshots_df[snapshots_df['is_forward_filled'] == True])
        print(f"  Real events: {real_events}")
        print(f"  Forward-filled: {ff_events}")
    
    print(f"\nTime range: {snapshots_df['datetime'].min()} to {snapshots_df['datetime'].max()}")
    snapshots_df.head()

## 3. Mid-Price Time Series

In [ ]:
if not snapshots_df.empty and 'mid_price' in snapshots_df.columns:
    fig = go.Figure()
    
    # Plot real events
    if 'is_forward_filled' in snapshots_df.columns:
        real_df = snapshots_df[snapshots_df['is_forward_filled'] == False]
        ff_df = snapshots_df[snapshots_df['is_forward_filled'] == True]
        
        fig.add_trace(go.Scatter(
            x=real_df['datetime'],
            y=real_df['mid_price'],
            mode='markers',
            name='Real Events',
            marker=dict(size=6, color='blue')
        ))
        
        # Forward-filled as line
        fig.add_trace(go.Scatter(
            x=ff_df['datetime'],
            y=ff_df['mid_price'],
            mode='lines',
            name='Forward-Filled',
            line=dict(color='lightblue', width=1),
            opacity=0.5
        ))
    else:
        fig.add_trace(go.Scatter(
            x=snapshots_df['datetime'],
            y=snapshots_df['mid_price'],
            mode='lines+markers',
            name='Mid Price'
        ))
    
    fig.update_layout(
        title=f"Mid-Price Over Time<br><sub>{MARKET_QUESTION}</sub>",
        xaxis_title="Time",
        yaxis_title="Mid Price ($)",
        hovermode='x unified',
        template='plotly_white'
    )
    fig.show()
else:
    print("No mid_price data available")

## 4. Bid-Ask Spread Analysis

In [ ]:
if not snapshots_df.empty and all(col in snapshots_df.columns for col in ['best_bid', 'best_ask', 'spread']):
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=('Best Bid vs Best Ask', 'Spread')
    )
    
    # Best bid and ask
    fig.add_trace(
        go.Scatter(x=snapshots_df['datetime'], y=snapshots_df['best_bid'],
                   name='Best Bid', line=dict(color='green')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=snapshots_df['datetime'], y=snapshots_df['best_ask'],
                   name='Best Ask', line=dict(color='red')),
        row=1, col=1
    )
    
    # Spread
    fig.add_trace(
        go.Scatter(x=snapshots_df['datetime'], y=snapshots_df['spread'],
                   name='Spread', fill='tozeroy', line=dict(color='purple')),
        row=2, col=1
    )
    
    fig.update_layout(
        title=f"Bid-Ask Spread Analysis<br><sub>{MARKET_QUESTION}</sub>",
        height=600,
        hovermode='x unified',
        template='plotly_white'
    )
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Spread ($)", row=2, col=1)
    fig.show()
else:
    print("Missing bid/ask/spread columns")

## 5. Orderbook Depth Visualization

In [ ]:
def parse_orderbook_levels(bids_json, asks_json):
    """Parse bids and asks JSON into DataFrames."""
    bids = pd.DataFrame(bids_json) if bids_json else pd.DataFrame(columns=['price', 'size'])
    asks = pd.DataFrame(asks_json) if asks_json else pd.DataFrame(columns=['price', 'size'])
    return bids, asks

if not snapshots_df.empty and 'bids' in snapshots_df.columns and 'asks' in snapshots_df.columns:
    # Get the latest snapshot for depth chart
    latest = snapshots_df.iloc[-1]
    bids_df, asks_df = parse_orderbook_levels(latest['bids'], latest['asks'])
    
    if not bids_df.empty or not asks_df.empty:
        fig = go.Figure()
        
        # Calculate cumulative depth
        if not bids_df.empty:
            bids_df = bids_df.sort_values('price', ascending=False)
            bids_df['cumulative'] = bids_df['size'].astype(float).cumsum()
            fig.add_trace(go.Scatter(
                x=bids_df['price'].astype(float),
                y=bids_df['cumulative'],
                name='Bids',
                fill='tozeroy',
                line=dict(color='green')
            ))
        
        if not asks_df.empty:
            asks_df = asks_df.sort_values('price', ascending=True)
            asks_df['cumulative'] = asks_df['size'].astype(float).cumsum()
            fig.add_trace(go.Scatter(
                x=asks_df['price'].astype(float),
                y=asks_df['cumulative'],
                name='Asks',
                fill='tozeroy',
                line=dict(color='red')
            ))
        
        fig.update_layout(
            title=f"Orderbook Depth (Latest Snapshot)<br><sub>{latest['datetime']}</sub>",
            xaxis_title="Price ($)",
            yaxis_title="Cumulative Size",
            template='plotly_white'
        )
        fig.show()
    else:
        print("No orderbook levels in latest snapshot")
else:
    print("No bids/asks data available")

## 6. Depth Over Time (Bid vs Ask Depth)

In [ ]:
if not snapshots_df.empty and 'bid_depth' in snapshots_df.columns and 'ask_depth' in snapshots_df.columns:
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=snapshots_df['datetime'],
        y=snapshots_df['bid_depth'],
        name='Bid Depth',
        fill='tozeroy',
        line=dict(color='green')
    ))
    
    fig.add_trace(go.Scatter(
        x=snapshots_df['datetime'],
        y=snapshots_df['ask_depth'],
        name='Ask Depth',
        fill='tozeroy',
        line=dict(color='red')
    ))
    
    fig.update_layout(
        title=f"Orderbook Depth Over Time<br><sub>{MARKET_QUESTION}</sub>",
        xaxis_title="Time",
        yaxis_title="Total Depth ($)",
        hovermode='x unified',
        template='plotly_white'
    )
    fig.show()
else:
    print("No depth data available")

## 7. Load and Visualize Trades

In [ ]:
# Fetch trades
trades_response = supabase.table("trades") \
    .select("*") \
    .eq("asset_id", TOKEN_ID) \
    .gte("timestamp", start_time) \
    .order("timestamp", desc=False) \
    .execute()

trades_df = pd.DataFrame(trades_response.data)
print(f"Loaded {len(trades_df)} trades")

if not trades_df.empty:
    trades_df['datetime'] = pd.to_datetime(trades_df['timestamp'], unit='ms')
    trades_df = trades_df.sort_values('timestamp')
    trades_df.head(10)

In [ ]:
if not trades_df.empty and 'price' in trades_df.columns:
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=('Trade Price', 'Trade Size'),
        row_heights=[0.6, 0.4]
    )
    
    # Color by side if available
    if 'side' in trades_df.columns:
        colors = trades_df['side'].map({'buy': 'green', 'sell': 'red'}).fillna('gray')
    else:
        colors = 'blue'
    
    # Trade prices
    fig.add_trace(
        go.Scatter(
            x=trades_df['datetime'],
            y=trades_df['price'],
            mode='markers',
            name='Trades',
            marker=dict(size=8, color=colors)
        ),
        row=1, col=1
    )
    
    # Trade sizes as bars
    if 'size' in trades_df.columns:
        fig.add_trace(
            go.Bar(
                x=trades_df['datetime'],
                y=trades_df['size'],
                name='Size',
                marker_color=colors if isinstance(colors, str) else colors.tolist()
            ),
            row=2, col=1
        )
    
    fig.update_layout(
        title=f"Trade Activity<br><sub>{MARKET_QUESTION}</sub>",
        height=600,
        hovermode='x unified',
        template='plotly_white',
        showlegend=False
    )
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Size", row=2, col=1)
    fig.show()
else:
    print("No trade data to visualize")

## 8. OHLCV Candlestick Chart

In [ ]:
# Build OHLCV candles from mid-price data
CANDLE_INTERVAL = '1min'  # Options: '1min', '5min', '15min', etc.

if not snapshots_df.empty and 'mid_price' in snapshots_df.columns:
    # Filter to real events only for cleaner OHLCV
    if 'is_forward_filled' in snapshots_df.columns:
        ohlcv_source = snapshots_df[snapshots_df['is_forward_filled'] == False].copy()
    else:
        ohlcv_source = snapshots_df.copy()
    
    if not ohlcv_source.empty:
        ohlcv_source = ohlcv_source.set_index('datetime')
        
        # Resample to OHLCV
        ohlcv = ohlcv_source['mid_price'].resample(CANDLE_INTERVAL).ohlc()
        ohlcv = ohlcv.dropna()
        
        # Add volume from depth changes if available
        if 'bid_depth' in ohlcv_source.columns:
            volume = ohlcv_source['bid_depth'].resample(CANDLE_INTERVAL).sum()
            ohlcv['volume'] = volume
        
        print(f"Generated {len(ohlcv)} candles at {CANDLE_INTERVAL} interval")
        ohlcv.head()

In [ ]:
if 'ohlcv' in dir() and not ohlcv.empty:
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.7, 0.3]
    )
    
    # Candlestick
    fig.add_trace(
        go.Candlestick(
            x=ohlcv.index,
            open=ohlcv['open'],
            high=ohlcv['high'],
            low=ohlcv['low'],
            close=ohlcv['close'],
            name='OHLC'
        ),
        row=1, col=1
    )
    
    # Volume bars if available
    if 'volume' in ohlcv.columns:
        colors = ['green' if c >= o else 'red' 
                  for o, c in zip(ohlcv['open'], ohlcv['close'])]
        fig.add_trace(
            go.Bar(
                x=ohlcv.index,
                y=ohlcv['volume'],
                name='Volume',
                marker_color=colors
            ),
            row=2, col=1
        )
    
    fig.update_layout(
        title=f"OHLCV Candlestick ({CANDLE_INTERVAL})<br><sub>{MARKET_QUESTION}</sub>",
        height=700,
        xaxis_rangeslider_visible=False,
        template='plotly_white'
    )
    fig.update_yaxes(title_text="Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.show()
else:
    print("No OHLCV data generated")

## 9. Compare Multiple Markets

In [ ]:
# Compare Up vs Down outcomes for the same 15m market
if not markets_df.empty:
    # Group by question to find Up/Down pairs
    # For 15m markets, compare recent Bitcoin markets
    btc_markets = markets_df[markets_df['question'].str.contains('Bitcoin', case=False, na=False)].copy()
    
    if btc_markets.empty:
        btc_markets = markets_df.head(4)
    
    fig = go.Figure()
    
    for _, market in btc_markets.head(6).iterrows():
        token_id = market['token_id']
        outcome = market.get('outcome', 'Unknown')
        question_short = market.get('question', '')[:40]
        label = f"{outcome} - {question_short}..."
        
        # Fetch snapshots for this market
        response = supabase.table("orderbook_snapshots") \
            .select("timestamp,mid_price") \
            .eq("asset_id", token_id) \
            .gte("timestamp", start_time) \
            .order("timestamp", desc=False) \
            .limit(1000) \
            .execute()
        
        if response.data:
            df = pd.DataFrame(response.data)
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            # Color by outcome
            color = 'green' if outcome in ['Up', 'Yes', 'Over'] else 'red' if outcome in ['Down', 'No', 'Under'] else None
            
            fig.add_trace(go.Scatter(
                x=df['datetime'],
                y=df['mid_price'],
                name=label,
                mode='lines',
                line=dict(color=color) if color else None
            ))
    
    fig.update_layout(
        title="Bitcoin 15m Markets - Up vs Down Comparison",
        xaxis_title="Time",
        yaxis_title="Mid Price ($)",
        hovermode='x unified',
        template='plotly_white',
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, font=dict(size=10))
    )
    fig.show()
else:
    print("No markets available for comparison")

## 10. Liquidity Heatmap

In [ ]:
def build_liquidity_heatmap(df, time_bins=50, price_bins=20):
    """Build a heatmap showing liquidity distribution over time and price."""
    heatmap_data = []
    
    for _, row in df.iterrows():
        timestamp = row['datetime']
        
        # Process bids
        if row.get('bids'):
            for level in row['bids']:
                heatmap_data.append({
                    'datetime': timestamp,
                    'price': float(level['price']),
                    'size': float(level['size']),
                    'side': 'bid'
                })
        
        # Process asks
        if row.get('asks'):
            for level in row['asks']:
                heatmap_data.append({
                    'datetime': timestamp,
                    'price': float(level['price']),
                    'size': float(level['size']),
                    'side': 'ask'
                })
    
    return pd.DataFrame(heatmap_data)

if not snapshots_df.empty and 'bids' in snapshots_df.columns:
    # Sample for performance
    sample_df = snapshots_df.iloc[::max(1, len(snapshots_df)//100)]
    liquidity_df = build_liquidity_heatmap(sample_df)
    
    if not liquidity_df.empty:
        fig = px.density_heatmap(
            liquidity_df,
            x='datetime',
            y='price',
            z='size',
            histfunc='sum',
            color_continuous_scale='Viridis',
            nbinsx=50,
            nbinsy=30
        )
        
        fig.update_layout(
            title=f"Liquidity Heatmap<br><sub>{MARKET_QUESTION}</sub>",
            xaxis_title="Time",
            yaxis_title="Price ($)",
            template='plotly_white'
        )
        fig.show()
    else:
        print("No liquidity data to visualize")
else:
    print("No orderbook data for heatmap")

## 11. Summary Statistics

In [ ]:
if not snapshots_df.empty:
    stats = {
        'Total Snapshots': len(snapshots_df),
        'Time Range': f"{snapshots_df['datetime'].min()} to {snapshots_df['datetime'].max()}",
        'Duration': str(snapshots_df['datetime'].max() - snapshots_df['datetime'].min()),
    }
    
    if 'mid_price' in snapshots_df.columns:
        stats.update({
            'Min Mid Price': f"${snapshots_df['mid_price'].min():.4f}",
            'Max Mid Price': f"${snapshots_df['mid_price'].max():.4f}",
            'Avg Mid Price': f"${snapshots_df['mid_price'].mean():.4f}",
            'Price Std Dev': f"${snapshots_df['mid_price'].std():.4f}",
        })
    
    if 'spread' in snapshots_df.columns:
        stats.update({
            'Avg Spread': f"${snapshots_df['spread'].mean():.4f}",
            'Max Spread': f"${snapshots_df['spread'].max():.4f}",
        })
    
    if 'is_forward_filled' in snapshots_df.columns:
        real = len(snapshots_df[snapshots_df['is_forward_filled'] == False])
        stats['Real Events'] = real
        stats['Forward-Filled'] = len(snapshots_df) - real
    
    stats_df = pd.DataFrame.from_dict(stats, orient='index', columns=['Value'])
    stats_df

In [ ]:
# Trade statistics
if not trades_df.empty:
    trade_stats = {
        'Total Trades': len(trades_df),
        'Time Range': f"{trades_df['datetime'].min()} to {trades_df['datetime'].max()}",
    }
    
    if 'price' in trades_df.columns:
        trade_stats.update({
            'Min Trade Price': f"${trades_df['price'].min():.4f}",
            'Max Trade Price': f"${trades_df['price'].max():.4f}",
            'Avg Trade Price': f"${trades_df['price'].mean():.4f}",
        })
    
    if 'size' in trades_df.columns:
        trade_stats.update({
            'Total Volume': f"{trades_df['size'].sum():.2f}",
            'Avg Trade Size': f"{trades_df['size'].mean():.4f}",
        })
    
    if 'side' in trades_df.columns:
        trade_stats['Buy Trades'] = len(trades_df[trades_df['side'] == 'buy'])
        trade_stats['Sell Trades'] = len(trades_df[trades_df['side'] == 'sell'])
    
    trade_stats_df = pd.DataFrame.from_dict(trade_stats, orient='index', columns=['Value'])
    trade_stats_df
else:
    print("No trades to summarize")